## <img src="https://lh3.googleusercontent.com/mUTbNK32c_DTSNrhqETT5aQJYFKok2HB1G2nk2MZHvG5bSs0v_lmDm_ArW7rgd6SDGHXo0Ak2uFFU96X6Xd0GQ=w160-h128" width="45" valign="top" alt="BigQuery"> Using GenAI (Gemini Pro and Imagen2) to create a Marketing Campaign


### License

In [ ]:
##################################################################################
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     https://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
###################################################################################

### Notebook Overview

**Goal:**
- Create a marketing campaign that will be email to customers.  The campaign should be a HTML email with a picture of the coffee/food.

**Steps:**
1. Determine the items we want to sell from our menu table.
2. Create a basic LLM prompt for Imagen 2 to render the image.
3. We want a really good image that inspires our customers to purchase our coffee/food so we will brainstorm with the LLM
   - Ask Gemini Pro to take our original image prompt and create 10 new creative prompts.
   - We are using the LLM to write our prompt for us.
   - We will also add in a prompt that has nothing to do with coffee/food ()
4. We want to verify the image contains the original coffee/food items we requested.
   - The LLM might have been too creative so let's verify each image.
   - Pass each image to Gemini Pro Vision and ask it to verify the contents of the image and explain its reasoning.
5. For each of the verified images, let's do a taste test.
   - Pass all the images to Gemini Pro Vision and ask it to rate each image from 1 to 100.
   - Rate the images based upon how realistic they are and creative.
   - Have the LLM explain its reasoning
6. We now have the highest rated image and we can see it compared to the original image.
7. Now, we will generate our marketing campaign.  
   - We will pass our prompt and the image to Gemini Pro Vision and ask it generate a marketing campaign.
   - We will do this for each City
   - We will ask the LLM to write the campaign in the native language for each city (English, Japanese)
8. With the marketing text we will now need it formatted as HTML
   - Pass the marketing text to ask Gemini to create HTML
   - Ask Gemini to style the email
   - Ask Gemini to leave a placeholder for our image
9. With the HMTL now replace the placeholder with our image
   - The image is base64 encoded so we inline the image versus hosting it on a server.
10. Save everything to BigQuery
      - Save a record for each city
      - Save all of our prompts (we can use this later to learn which prompts generate the best results)
      - Save the HTML (UTF-8 for international character support)
      - Save the images to a GCS bucket

## Initialize Python

In [ ]:
project_id="${project_id}"
location="us-central1"
model_id = "imagegeneration@005"

# No need to set these
city_names=["New York City", "London", "Tokyo", "San Francisco"]
city_ids=[1,2,3,4]
city_languages=["American English", "British English", "Japanese", "American English"]
number_of_coffee_trucks = "4"

llm_marketing_prompt = []
llm_marketing_prompt_text = []

llm_marketing_prompt_html = []
llm_marketing_prompt_html_text = []

gcs_storage_bucket = "${data_beans_curated_bucket}"
gcs_storage_path = "data-beans/marketing-campaign/"

In [ ]:
from PIL import Image
from IPython.display import HTML
import IPython.display
import google.auth
import requests
import json
import uuid
import base64
import os
import cv2

from google.cloud import bigquery
client = bigquery.Client()

## Imagen2 / Gemini Pro / Gemini Pro Vision (Helper Functions)

#### Imagen2

In [ ]:
def ImageGen(prompt):
  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/image-generation
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project}/locations/{location}/publishers/google/models/imagegeneration:predict"

  payload = {
    "instances": [
      {
        "prompt": prompt
      }
    ],
    "parameters": {
      "sampleCount": 1
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    image_data = json.loads(response.content)["predictions"][0]["bytesBase64Encoded"]
    image_data = base64.b64decode(image_data)
    filename= str(uuid.uuid4()) + ".png"
    with open(filename, "wb") as f:
      f.write(image_data)
    print(f"Image generated OK.")
    return filename
  else:
    error = f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'"
    raise RuntimeError(error)

#### Gemini Pro LLM

In [ ]:
def GeminiProLLM(prompt, temperature = .8, topP = .8, topK = 40):

  if temperature < 0:
    temperature = 0

  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/gemini#gemini-pro
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/publishers/google/models/gemini-pro:streamGenerateContent"

  payload = {
    "contents": {
      "role": "user",
      "parts": {
          "text": prompt
      },
    },
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    },
    "generation_config": {
      "temperature": temperature,
      "topP": topP,
      "topK": topK,
      "maxOutputTokens": 8192,
      "candidateCount": 1
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    json_response = json.loads(response.content)
    llm_response = ""
    for item in json_response:
      try:
        llm_response = llm_response + item["candidates"][0]["content"]["parts"][0]["text"]
      except Exception as err:
        print(f"response.content: {response.content}")
        raise RuntimeError(err)

    # Remove some typically response characters (if asking for a JSON reply)
    llm_response = llm_response.replace("```json","")
    llm_response = llm_response.replace("```","")

    # print(f"llm_response:\n{llm_response}")
    return llm_response
  else:
    error = f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'"
    raise RuntimeError(error)


#### Gemini Pro Vision LLM

In [ ]:
# Use the Gemini with Vision
def GeminiProVisionLLM(prompt, imageBase64, temperature = .4, topP = 1, topK = 32):

  if temperature < 0:
    temperature = 0

  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/gemini#gemini-pro
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/publishers/google/models/gemini-pro-vision:streamGenerateContent"

  payload = {
  "contents": [
      {
        "role": "user",
        "parts": [
          {
            "text": prompt
          },
          {
            "inlineData": {
              "mimeType": "image/png",
              "data": f"{imageBase64}"
            }
          }
        ]
      }
    ],
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    },
    "generation_config": {
      "temperature": temperature,
      "topP": topP,
      "topK": topK,
      "maxOutputTokens": 2048,
      "candidateCount": 1
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    json_response = json.loads(response.content)
    llm_response = ""
    for item in json_response:
      llm_response = llm_response + item["candidates"][0]["content"]["parts"][0]["text"]

    # Remove some typically response characters (if asking for a JSON reply)
    llm_response = llm_response.replace("```json","")
    llm_response = llm_response.replace("```","")

    # print(f"llm_response:\n{llm_response}")
    return llm_response
  else:
    error = f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'"
    raise RuntimeError(error)


In [ ]:
# Use the Gemini with Vision
def GeminiProVisionMultipleFileLLM(prompt, image_prompt, temperature = .4, topP = 1, topK = 32):
  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/gemini#gemini-pro
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/publishers/google/models/gemini-pro-vision:streamGenerateContent"


  parts = []
  new_item = {
      "text": prompt
      }
  parts.append(new_item)

  for item in image_prompt:
    new_item = {
        "text": f"Image Name: {item['llm_image_filename']}:\n"
        }
    parts.append(new_item)
    new_item = {
        "inlineData": {
            "mimeType": "image/png",
            "data": item["llm_image_base64"]
            }
        }
    parts.append(new_item)

  payload = {
  "contents": [
      {
        "role": "user",
        "parts": parts
      }
    ],
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    },
    "generation_config": {
      "temperature": temperature,
      "topP": topP,
      "topK": topK,
      "maxOutputTokens": 2048,
      "candidateCount": 1
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    json_response = json.loads(response.content)
    llm_response = ""
    for item in json_response:
      llm_response = llm_response + item["candidates"][0]["content"]["parts"][0]["text"]

    # Remove some typically response characters (if asking for a JSON reply)
    llm_response = llm_response.replace("```json","")
    llm_response = llm_response.replace("```","")

    # print(f"llm_response:\n{llm_response}")
    return llm_response
  else:
    error = f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'"
    raise RuntimeError(error)


#### Helper Functions

In [ ]:
def convert_png_to_base64(image_path):
  image = cv2.imread(image_path)

  # Convert the image to a base64 string.
  _, buffer = cv2.imencode('.png', image)
  base64_string = base64.b64encode(buffer).decode('utf-8')

  return base64_string

In [ ]:
def RunQuery(sql):
  import time

  if (sql.startswith("SELECT") or sql.startswith("WITH")):
      df_result = client.query(sql).to_dataframe()
      return df_result
  else:
    job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
    query_job = client.query(sql, job_config=job_config)

    # Check on the progress by getting the job's updated state.
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
    )
    print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    while query_job.state != "DONE":
      time.sleep(2)
      query_job = client.get_job(
          query_job.job_id, location=query_job.location
          )
      print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    if query_job.error_result == None:
      return True
    else:
      return False

In [ ]:
def GetNextPrimaryKey(fully_qualified_table_name, field_name):
  sql = f"""
  SELECT IFNULL(MAX({field_name}),0) AS result
    FROM `{fully_qualified_table_name}`
  """
  # print(sql)
  df_result = client.query(sql).to_dataframe()
  # display(df_result)
  return df_result['result'].iloc[0] + 1

In [ ]:
# This was generated by GenAI

def copy_file_to_gcs(local_file_path, bucket_name, destination_blob_name):
  """Copies a file from a local drive to a GCS bucket.

  Args:
      local_file_path: The full path to the local file.
      bucket_name: The name of the GCS bucket to upload to.
      destination_blob_name: The desired name of the uploaded file in the bucket.

  Returns:
      None
  """

  import os
  from google.cloud import storage

  # Ensure the file exists locally
  if not os.path.exists(local_file_path):
      raise FileNotFoundError(f"Local file '{local_file_path}' not found.")

  # Create a storage client
  storage_client = storage.Client()

  # Get a reference to the bucket
  bucket = storage_client.bucket(bucket_name)

  # Create a blob object with the desired destination path
  blob = bucket.blob(destination_blob_name)

  # Upload the file from the local filesystem
  content_type = ""
  if local_file_path.endswith(".html"):
    content_type = "text/html; charset=utf-8"

  if local_file_path.endswith(".json"):
    content_type = "application/json; charset=utf-8"

  if content_type == "":
    blob.upload_from_filename(local_file_path)
  else:
    blob.upload_from_filename(local_file_path, content_type = content_type)

  print(f"File '{local_file_path}' uploaded to GCS bucket '{bucket_name}' as '{destination_blob_name}.  Content-Type: {content_type}'.")

## Generating a Marketing Campaign for each City

#### Let's start with a basic Vision prompt that a Human might create.
- This is a basic prompt that someone might create

In [ ]:
# This will generate a fairly basic image that might not be the most inviting image

from random import randrange
random_1_to_5 = randrange(5) + 1;  

# you can hard code this for a specific value
random_1_to_5 = 5 

match random_1_to_5:
  case 1:
        picture_description = "Coffee Glazed Donut with Java Chip Pancakes"
  case 2:
        picture_description = "Cappuccino Cheesecake with a Cinnamon Spice latte"
  case 3:
        picture_description = "Coffee-Infused Chocolate Truffles with Coffee Bean Ice Cream"
  case 4:
        picture_description = "Coffee Tiramisu with Cafe Con Leche"
  case 5:
        picture_description = "Mocha Pancakes with Vanilla Bean ice cream"
  case _:
      picture_description = "Coffee Glazed Donut with Java Chip Pancakes"

print(f"Picture Description: {picture_description}")

llm_human_image_prompt = f"Create an image that contains a {picture_description} that I can use for a marketing campaign."
human_prompt_filename = ImageGen(llm_human_image_prompt)
img = Image.open(human_prompt_filename)
llm_human_image_json = {
    "gcs_storage_bucket" : gcs_storage_bucket,
    "gcs_storage_path" : gcs_storage_path,
    "llm_image_filename" : human_prompt_filename
}
img.thumbnail([500,500])
IPython.display.display(img)

#### Now let's improve our image by calling Gemini Pro create a better Imagen 2 prompt
- LLMs can be used to create LLM prompts for better results.
- We are basically asking the LLM to write the prompt that we will then submit as our Imagen 2 prompt.

In [ ]:
example_return_json = '[ { "prompt" : "response 1" }, { "prompt" : "response 2" }, { "prompt" : "A prompt for good LLMs" }, { "prompt" : "Generate an image that is awesome" }]'

llm_generated_image_prompt=f"""For the below prompt, rewrite it in 10 different ways so I get the most creative images to be generated.
Try creative thinking, generate innovative and out-of-the-box ideas to solve the problem.
Explore unconventional solutions, thinking beyond traditional boundaries, and encouraging imagination and originality.
Embrace unconventional ideas and mutate the prompt in a way that surprises and inspires unique variations.
Think outside the box and develop a mutator prompt that encourages unconventional approaches and fresh perspectives.
Return the results in below "JSON format" with no special characters, headers or formatting.
Limit each json result to 256 characters.

JSON format:
{example_return_json}

Prompt: "Create a picture of a {picture_description} to be used for a marketing campaign."
"""

llm_success = False
temperature=.8
while llm_success == False:
  try:
    llm_response = GeminiProLLM(llm_generated_image_prompt, temperature=temperature, topP=.8, topK = 40)
    print(llm_response)
    bracket_index = llm_response.find('[')
    if bracket_index > 0:
      print("Removing LLM text that preceeds the JSON")
      llm_response = llm_response[bracket_index:]
      print(llm_response)
    json.loads(llm_response)
    llm_success = True
  except:
    # Reduce the temperature for more accurate generation
    temperature = temperature - .05
    print("Regenerating...")

#### Generate the images with our enhanced prompts
- Use our generated prompts to pass to Imagen 2

In [ ]:
llm_json = json.loads(llm_response)

# Add an image to the generation that will not contain any food items.  We will test this later.
llm_json.append({'prompt' : 'Draw a coffee truck with disco lights.'})

image_files = []

for item in llm_json:
  print(item["prompt"])
  try:
    image_file = ImageGen(item["prompt"])
    image_files.append ({
        "llm_prompt" : item["prompt"],
        "gcs_storage_bucket" : gcs_storage_bucket,
        "gcs_storage_path" : gcs_storage_path,
        "llm_image_filename" : image_file,
        "llm_validated_by_llm" : False
    })
  except:
    print("Image failed to generate.")

In [ ]:
# View the results

llm_generated_image_json = []

for item in image_files:
  # Json for BigQuery
  llm_generated_image_json.append( {
      "llm_prompt" : item["llm_prompt"],
      "gcs_storage_bucket" : item["gcs_storage_bucket"],
      "gcs_storage_path" : item["gcs_storage_path"],
      "llm_image_filename" : item["llm_image_filename"]
      })

  print(f"Filename: {item['llm_image_filename']}")
  img = Image.open(item["llm_image_filename"])
  img.thumbnail([500,500])
  IPython.display.display(img)

#### Verify that each image contains the food items we asked for
- Is the image an image that actaully represents what we asked for?
- Sometimes the image might not contain the details we asked for.
- We will run each image through Gemini Pro Vision and verify it is actaully what we want.

In [ ]:
example_return_json = '{ "response" : true, "explanation" : "Reason why..." }'

llm_validated_image_json = []
number_of_valid_images = 0

llm_validated_image_prompt = f"""Is attached image a picture of "{picture_description}"?
Respond with a boolean of true or false and place in the "response" field.
Explain your reasoning for each image and place in the "explanation" field.
Return the results in JSON with no special characters or formatting.

Place the results in the following JSON structure:
{example_return_json}
"""

for item in image_files:
  print(f"LLM Prompt : {item['llm_prompt']}")
  print(f"LLM Filename : {item['llm_image_filename']}")
  imageBase64 = convert_png_to_base64(item['llm_image_filename'])

  llm_success = False
  temperature = .4
  while llm_success == False:
    try:
      llm_response = GeminiProVisionLLM(llm_validated_image_prompt, imageBase64, temperature=temperature)
      print(f"llm_response : {llm_response}")
      llm_json = json.loads(llm_response)
      llm_success = True
    except:
      # Reduce the temperature for more accurate generation
      temperature = temperature - .05
      print("Regenerating...")

  # Mark this item as useable
  if llm_json["response"] == True:
    item["llm_validated_by_llm"] = True
    number_of_valid_images = number_of_valid_images + 1

  item["llm_validated_by_llm_explanation"] = llm_json["explanation"]

  llm_validated_image_json.append( {
      "llm_prompt" : item["llm_prompt"],
      "gcs_storage_bucket" : item["gcs_storage_bucket"],
      "gcs_storage_path" : item["gcs_storage_path"],
      "llm_image_filename" : item["llm_image_filename"],
      "llm_validated_by_llm" : item["llm_validated_by_llm"],
      "llm_validated_by_llm_explanation" : item["llm_validated_by_llm_explanation"]
      })

#### Now let's ask the LLM which one looks the tastiest
- We want the image that looks the best so we will rate each one from 1 to 10
- Add all the verified images to a Gemini Pro Vision prompt.  Gemini supports multimodal prompts.
- This will help us narrow down which image we will use for the marketing campaign

In [ ]:
image_prompt = []

for item in image_files:
  if item["llm_validated_by_llm"] == True:
    print(f"Adding image {item['llm_image_filename']} to taste test.")
    imageBase64 = convert_png_to_base64(item['llm_image_filename'])
    new_item = {
        "llm_image_filename" : item['llm_image_filename'],
        "llm_image_base64" : f"{imageBase64}"
    }
    image_prompt.append(new_item)

example_return_json='[ {"image_name" : "name", "rating" : 10, "explanation": ""}]'

llm_taste_test_image_prompt=f"""You are going to be presented with {number_of_valid_images}.
You must critique each image and assign it a score from 1 to 100.
You should compare the images to one another.
You should evaluate each image multiple times.

Score each image based upon the following:
- Appetizing images should get a high rating.
- Realistic images should get a high rating.
- Thought provoking images should get a high rating.
- Plastic looking images should get a low rating.
- Abstract images should get a low rating.

Think the rating through step by step for each image.

Place the result of the scoring process in the "rating" field.
Return the image name and place in the "image_name" field.
Explain your reasoning and place in the "explanation" field in less than 20 words.

Place the results in the following JSON structure:
{example_return_json}
"""

llm_success = False
temperature=1
while llm_success == False:
  try:
    llm_response = GeminiProVisionMultipleFileLLM(llm_taste_test_image_prompt, image_prompt, temperature = temperature)
    llm_success = True
  except:
    # Reduce the temperature for more accurate generation
    temperature = temperature - .05
    print("Regenerating...")


print(f"llm_response : {llm_response}")

In [ ]:
# Show the highest rated image (and save the id)
llm_json = json.loads(llm_response)
highest_rated_llm_image_rating = 0
highest_rated_llm_image_filename = ""
llm_taste_test_image_json = []

for item in llm_json:
  # Find and update each image with the rating and explanation
  for existing in image_files:
    if existing["llm_image_filename"] == item["image_name"]:
      existing["image_rating"] = item["rating"]
      existing["image_explanation"] = item["explanation"]
      llm_taste_test_image_json.append( {
          "llm_prompt" : existing["llm_prompt"],
          "gcs_storage_bucket" : existing["gcs_storage_bucket"],
          "gcs_storage_path" : existing["gcs_storage_path"],
          "llm_image_filename" : existing["llm_image_filename"],
          "llm_validated_by_llm" : existing["llm_validated_by_llm"],
          "llm_validated_by_llm_explanation" : existing["llm_validated_by_llm_explanation"],
          "image_rating" : existing["image_rating"],
          "image_rating_explanation" : existing["image_explanation"]
          })
      if existing["image_rating"] >= highest_rated_llm_image_rating:
        highest_rated_llm_image_rating = existing["image_rating"]
        highest_rated_llm_image_filename = existing["llm_image_filename"]
      break


print(f"Highest rated highest_rated_llm_image_filename: {highest_rated_llm_image_filename}")
img = Image.open(highest_rated_llm_image_filename)
img.thumbnail([500,500])
IPython.display.display(img)

In [ ]:
# Compare this with our human generated prompt image
img = Image.open(human_prompt_filename)
img.thumbnail([500,500])
IPython.display.display(img)

#### Now that we have our image, let's generate the best marketing text

In [ ]:
example_return_json_1 = '{ "city" : "New York City", "subject" : "email subject" , "body" : "email body" }'
example_return_json_2 = '{ "city" : "London", "subject" : "marketing subject" , "body" : "The body of the email message" }'
imageBase64 = convert_png_to_base64(highest_rated_llm_image_filename)

llm_marketing_campaign = []

# Loop for each city
for city_index in range(0, 4):
  print(f"Processing city: {city_names[city_index]}")

  prompt=f"""You run a fleet of coffee trucks in {city_names[city_index]}.
  We need to craft a compelling email marketing campaign for the below image.
  Embrace unconventional ideas and thinking that surprises and inspires unique variations.
  We want to offer exclusive discounts or early access to drive sales.
  Look at the image and come up with a savory message.
  The image contains "{picture_description}".

  The marketing campaign should per personalize for {city_names[city_index]}.
  Do not mention the weather in the message.
  Write the response in using the language "{city_languages[city_index]}".
  The campaign should be less than 1500 characters.
  The email message should be formatted in text, not HTML.
  Sign the email message "Sincerely, Data Beans".
  Mention that people can find the closest coffee truck location by using our mobile app.

  Return the results in JSON with no special characters or formatting.
  Double check for special characters especially for Japanese.
  Place the value of "{city_names[city_index]}" in the "city" field of the JSON response.

  Example Return Data:
  {example_return_json_1}
  {example_return_json_2}

  Image of items we want to sell:
  """

  llm_success = False
  temperature = .8
  while llm_success == False:
    try:
      llm_response = GeminiProVisionLLM(prompt, imageBase64, temperature=.8, topP=.9, topK = 40)
      llm_json = json.loads(llm_response)
      print(llm_response)
      llm_success = True
    except:
      # Reduce the temperature for more accurate generation
      temperature = temperature - .05
      print("Regenerating...")

  llm_marketing_campaign.append(llm_json)

  llm_marketing_prompt.append(prompt)
  llm_marketing_prompt_text.append(llm_response)

#### Create the HTML email message we will send to our customers
- We do not know html
- Also, it would be nice to base64 encode the image into the stylesheet

In [ ]:
imageBase64 = convert_png_to_base64(highest_rated_llm_image_filename)

for item in llm_marketing_campaign:
  print (f'City:    {item["city"]}')
  print (f'Subject: {item["subject"]}')

  prompt=f"""Convert the below text into a well-structured HTML document.
Refrain from using <h1>, <h2>, <h3>, and <h4> tags.
Create inline styles.
Make the styles fun, eye catching and exciting.
Use "Helvetica Neue" as the font.
All text should be left aligned.
Avoid fonts larger than 16 pixels.
Do not change the language.  Keep the text in the native language.
Do not include any notes, only generate the HTML.

Include the following image in the html:
- The image is located at: https://REPLACE_ME
- The image url should have a "width" of 500 and "height" of 500.

Double check that you did not use any <h1>, <h2>, <h3>, or <h4> tags.

Text:
{item["body"]}
"""

  # The LLM can generate some HTML tags that can cause issues when parsing the result
  llm_success = False
  temperature = .5

  while llm_success == False:
    try:
      llm_response = GeminiProLLM(prompt, temperature=temperature, topP=.5, topK = 20)

      if llm_response.startswith("html"):
        llm_response = llm_response[4:] # incase the response is formatted like markdown

      # Replace the image with an inline image (this avoids a SignedURL or public access to GCS bucket)
      html_text = llm_response.replace("https://REPLACE_ME", f"data:image/png;base64, {imageBase64}")
      item["html"] = html_text

      #print (f'Html:    {item["html"]}')
      filename= str(item["city"]).replace(" ","-") + ".html"
      with open(filename, "w", encoding='utf8') as f:
        f.write(item["html"])
      print ("")

      llm_success = True
      llm_marketing_prompt_html.append(prompt)
      llm_marketing_prompt_html_text.append(html_text)

    except:
      # Reduce the temperature for more accurate generation
      temperature = temperature - .05
      print("Regenerating...")


In [ ]:
HTML(filename=str("New York City").replace(" ","-") + ".html")

In [ ]:
HTML(filename=str("London").replace(" ","-") + ".html")

In [ ]:
HTML(filename=str("Tokyo").replace(" ","-") + ".html")

In [ ]:
HTML(filename=str("San Francisco").replace(" ","-") + ".html")

#### Save the results to BigQuery

In [ ]:
%%bigquery
CREATE TABLE IF NOT EXISTS `${project_id}.${bigquery_data_beans_curated_dataset}.marketing_gen_ai_insight`
--CREATE OR REPLACE TABLE `${project_id}.${bigquery_data_beans_curated_dataset}.marketing_gen_ai_insight` -- only use to replace the entire table
(
    marketing_gen_ai_insight_id    INTEGER   NOT NULL OPTIONS(description="Primary key."),
    marketing_gen_ai_insight_type  STRING    NOT NULL OPTIONS(description="The type of insight: WEATHER, EVENT, CAMPAIGN, SOCIAL, WAIT-TIME, SALES, VIDEO-AI"),
    insight_datetime               TIMESTAMP NOT NULL OPTIONS(description="The datetime of the GenAI insight."),
    applies_to_entity_type         STRING    NOT NULL OPTIONS(description="The type of entity to which the insight applies: Company, City, Truck, Customer"),
    applies_to_entity_id           INTEGER   NOT NULL OPTIONS(description="The id (primary key) of entity to which the insight applies."),
    applies_to_entity_name         STRING    NOT NULL OPTIONS(description="The name to which the insight applies."),

    picture_description            STRING    NOT NULL OPTIONS(description="The description of the image we want to generate."),
    json_filename                  STRING    NOT NULL OPTIONS(description="The location of the json file that holds all the prompts."),
    html_filename                  STRING    NOT NULL OPTIONS(description="The location of the HTML file for the marketing campaign."),
    subject                        STRING    NOT NULL OPTIONS(description="The generated subject line for the marketing campaign."),
)
CLUSTER BY marketing_gen_ai_insight_id;

In [ ]:
from datetime import datetime

# Get the current date and time
now = datetime.now()

# Format the date and time as desired
formatted_date = now.strftime("%Y-%m-%d-%H-%M")

print(formatted_date)

In [ ]:
# Copy all image files to storage
for item in image_files:
  copy_file_to_gcs(item["llm_image_filename"],item["gcs_storage_bucket"], item["gcs_storage_path"] + formatted_date + "/Imagen2_" + item["llm_image_filename"])

In [ ]:
copy_file_to_gcs(llm_human_image_json["llm_image_filename"],llm_human_image_json["gcs_storage_bucket"], llm_human_image_json["gcs_storage_path"] + formatted_date + "/Imagen2_basic_prompt_" + item["llm_image_filename"])

In [ ]:
marketing_gen_ai_insight_id = GetNextPrimaryKey("${project_id}.${bigquery_data_beans_curated_dataset}.marketing_gen_ai_insight","marketing_gen_ai_insight_id")

In [ ]:
# Copy the html images
for city_index in range(0, 4):
  filename = city_names[city_index].replace(" ","-") + ".html"
  file_destination_name = formatted_date + "/" + str(marketing_gen_ai_insight_id + city_index) + "-" + city_names[city_index].replace(" ","-") + ".html"
  copy_file_to_gcs(filename,gcs_storage_bucket, gcs_storage_path + file_destination_name)

In [ ]:
# Output the full JSON
for city_index in range(0, 4):
  html_filename = "gs://" + gcs_storage_bucket + "/" + gcs_storage_path + formatted_date + "/" + str(marketing_gen_ai_insight_id + city_index) + "-" + city_names[city_index].replace(" ","-") + ".html"

  json_payload = {
      "llm_human_image_prompt" : llm_human_image_prompt,
      "llm_human_image_json" : llm_human_image_json,
      "llm_generated_image_prompt" : llm_generated_image_prompt,
      "llm_generated_image_json" : llm_generated_image_json,
      "llm_validated_image_prompt" : llm_validated_image_prompt,
      "llm_validated_image_json" : llm_validated_image_json,
      "llm_taste_test_image_prompt" : llm_taste_test_image_prompt,
      "llm_taste_test_image_json" : llm_taste_test_image_json,
      "llm_marketing_prompt" : llm_marketing_prompt[city_index],
      "llm_marketing_prompt_text" : llm_marketing_prompt_text[city_index],
      "llm_marketing_prompt_html" : llm_marketing_prompt_html[city_index],
      "llm_marketing_prompt_html_filename" : html_filename,
  }

  json_filename = str(marketing_gen_ai_insight_id + city_index) + "-" + city_names[city_index].replace(" ","-") + ".json"
  with open(json_filename, "w") as f:
    f.write(json.dumps(json_payload, indent=3))

  copy_file_to_gcs(json_filename,gcs_storage_bucket, gcs_storage_path + formatted_date + "/" +json_filename)


In [ ]:
for city_index in range(0, 4):
  print(f"Processing city: {city_names[city_index]}")

  json_filename = "gs://" + gcs_storage_bucket + "/" + \
    gcs_storage_path + formatted_date + "/" +str(marketing_gen_ai_insight_id) + '-' + str(city_ids[city_index]) + "-" + city_names[city_index].replace(" ","-") + ".json"

  html_filename = "gs://" + gcs_storage_bucket + "/" + \
    gcs_storage_path + formatted_date + "/" +str(marketing_gen_ai_insight_id) + '-' + str(city_ids[city_index]) + "-" + city_names[city_index].replace(" ","-") + ".html"

  subject = ""
  for item in llm_marketing_campaign:
    if item["city"] == city_names[city_index]:
      subject = item["subject"]
      break

  picture_description_string = picture_description.replace("'","\'")
  subject_string = subject.replace("'","\\'")

  sql=f"""INSERT INTO `${project_id}.${bigquery_data_beans_curated_dataset}.marketing_gen_ai_insight`
                      (marketing_gen_ai_insight_id, marketing_gen_ai_insight_type, insight_datetime,
                      applies_to_entity_type, applies_to_entity_id, applies_to_entity_name,
                      picture_description, json_filename, html_filename, subject)
              VALUES ({marketing_gen_ai_insight_id}, 'MARKETING-EMAIL', CURRENT_TIMESTAMP(),
                      'CITY', {city_ids[city_index]}, '{city_names[city_index]}',
                      '{picture_description_string}', '{json_filename}', '{html_filename}', '{subject_string}')"""

  print(sql)
  RunQuery(sql)
  marketing_gen_ai_insight_id = marketing_gen_ai_insight_id + 1
